# Serialization and __Composition__

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
) # 프롬프트를 여러 개 관리하기 위함

In [21]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonation a {character}
    """
)

example = PromptTemplate.from_template(
    """
    this is an example of how you talk:

    Huamn: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

AIMessage(content="Arrrr! That be a fine question, matey! I be lovin' me hearty sea biscuits and salted fish, aye! Nothing beats a good feast after a day of plunderin' the high seas!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 80, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano', 'system_fingerprint': 'fp_04d3664870', 'finish_reason': 'stop', 'logprobs': None}, id='run--3d251b0d-8dfc-483e-95fe-57627c23c2e9-0')

# __Serialization__ and Composition 

In [14]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
    streaming=True,
    callbacks= [
        StreamingStdOutCallbackHandler()
    ]
)

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

# prompt.format(country = "Germany")

In [15]:
prompt.invoke({
    "country" : "Germany"
})

StringPromptValue(text='What is the capital of Germany')

In [16]:
chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'service_tier': 'default'}, id='run--b6d82925-0e8d-476a-a1c3-ffd7a70c85d2-0', usage_metadata={'total_cost': 0})

# Caching

In [23]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)
chat = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

'The capital of Japan is Tokyo.'

In [18]:
chat.predict("What is the capital of Japan")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Japan"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of Japan is Tokyo.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of Japan is Tokyo.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 7,
                "prompt_tokens": 13,
                "total_tokens": 20,
                "completion_tokens_details": {
             

'The capital of Japan is Tokyo.'

In [19]:
chat.predict("What is the capital of Korea")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the capital of Korea"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The capital of South Korea is Seoul.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The capital of South Korea is Seoul.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 8,
                "prompt_tokens": 13,
                "total_tokens": 21,
                "completion_tokens_details": {
 

'The capital of South Korea is Seoul.'

# Serialization

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(False)

chat = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipt for beer")
    b = chat.predict("What is the recipt for chocolate")

    print(usage)
    print(usage.total_cost) # 전체비용
    print(usage.total_tokens) # 토큰 총합
    print(usage.prompt_tokens) #프롬프트에 사용된 토큰 수
    print(usage.completion_tokens) # 모델이 쓴 토큰

Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')
Error in OpenAICallbackHandler.on_llm_end callback: KeyError('total_tokens')


Tokens Used: 0
	Prompt Tokens: 0
		Prompt Tokens Cached: 0
	Completion Tokens: 0
		Reasoning Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
0.0
0
0
0
